In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns

In [28]:
# return a trimmed copy of the dataframe containing only the defence exp data
def trim_df(df):
    category_length = len(df) // 2
    df_copy= df.copy()
    
    trimmed_df = df_copy.iloc[:category_length, :2]
    
    return trimmed_df

# create a dataframe containing the successful number of hits, the sample size (n) and an estimator for p (p_hat)
# where p_hat is an estimate for the probability of hitting a 1 with either an Event RPG or Iron Scimitar
def get_probability_df(df, weapon_type):
    attack_speed = {   "Event RPG"   : 1.8,  # Event RPG attacks once every 1.8s 
                     "Iron Scimitar" : 2.4 } # Iron Scimitar attacks once every 2.4s
    
    successful_attacks = df.loc[1:, "Exp Gained"].to_numpy() / 4
    total_attacks = df.loc[1:, "Time (min)"].to_numpy() * 60 / attack_speed[weapon_type]
    total_attacks = [math.floor(attacks) for attacks in total_attacks]
    p_hat = successful_attacks / total_attacks
    
    d = { "Successful Attacks" : successful_attacks,
          "Total Attacks"      : total_attacks,
          "p hat"              : p_hat }
    
    probability_df = pd.DataFrame(data = d)
    probability_df["Weapon"] = weapon_type
    
    return probability_df

def display_parameters(n, p_hat, q):
    print("    n:", n)
    print("p_hat:", p_hat)
    print("    q:", q)

In [3]:
# load processed data
combined_df = pd.read_csv("data/combined_data.csv")

In [26]:
# transform data 
section_length = len(combined_df) // 4
trimmed_rpg_df = combined_df.iloc[:section_length, :].drop("Exp Type", axis = 1)
trimmed_scim_df = combined_df.iloc[section_length * 2 + 1:section_length * 3, :].drop("Exp Type", axis = 1)

# calculate and add the probabilities of hitting a 1 to the dataframes
rpg_probability_df = get_probability_df(trimmed_rpg_df, "Event RPG")
scim_probability_df = get_probability_df(trimmed_scim_df, "Iron Scimitar")
print(scim_probability_df)

# transform data for visualization
combined_probability_df = rpg_probability_df.append(scim_probability_df)
combined_probability_df_length = int(len(combined_probability_df))

    Successful Attacks  Total Attacks     p hat Weapon
0                 67.0            250  0.268000    NaN
1                120.0            500  0.240000    NaN
2                170.0            750  0.226667    NaN
3                219.0           1000  0.219000    NaN
4                263.0           1250  0.210400    NaN
5                319.0           1500  0.212667    NaN
6                374.0           1750  0.213714    NaN
7                422.0           2000  0.211000    NaN
8                482.0           2250  0.214222    NaN
9                533.0           2500  0.213200    NaN
10               584.0           2750  0.212364    NaN
11               634.0           3000  0.211333    NaN
12               695.0           3250  0.213846    NaN
13               751.0           3500  0.214571    NaN
14               807.0           3750  0.215200    NaN
15               855.0           4000  0.213750    NaN
16               911.0           4250  0.214353    NaN
17        

In [ ]:
# write processed data to csv
combined_probability_df.to_csv("data/combined_probability_data.csv", index = False)

In [ ]:
# load processed data
combined_probability_df = pd.read_csv("data/combined_probability_data.csv")

In [ ]:
# Event RPG sample size, p estimator and q
rpg_row = combined_probability_df_length // 2 - 1
rpg_n, rpg_p_hat = combined_probability_df.iloc[rpg_row, 1], round(combined_probability_df.iloc[rpg_row, 2], 6)
rpg_q = 1 - rpg_p_hat

print("--- Event RPG --- ")
display_parameters(rpg_n, rpg_p_hat, rpg_q)

In [ ]:
# Iron Scimitar sample size, p estimator and q
scim_row = combined_probability_df_length - 1
scim_n, scim_p_hat = combined_probability_df.iloc[scim_row, 1], round(combined_probability_df.iloc[scim_row, 2], 6)
scim_q = 1 - scim_p_hat

print("--- Iron Scimitar --- ")
display_parameters(scim_n, scim_p_hat, scim_q)

In [ ]:
# visualize trend of p_hat as sample size n increases 
success_scatter = sns.relplot(x = "Total Attacks" ,
                              y = "p hat",
                              hue = "Weapon",
                              data = combined_probability_df)

success_scatter.set(xlabel = "Total Attacks",
                    ylabel = "Probability of a Successful Hit")

plt.xlim(5, None)
plt.show()

In [ ]:
# graph normal distributions 

# n = 4000 for equal sample size 
# means equal to np with standard devidation npq 
# calculate 2P(Z >= test stat)
# test stat is (mu1 - mu2) / sqrt ((sd1 + sd2) / (n))
